In [1]:
import time
import numpy as np
import pandas as pd
import json
import seaborn as sb
from matplotlib import pyplot as plt
from matplotlib import rcParams
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
import seaborn as sb

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
import pickle

In [5]:
tutor_df = pd.read_pickle("./model_base_data.pkl")
tutor_df.head()

,qual_encoded,background_check,num_subjects,hours_tutoring,edu_length,mean_hours,state_cb,qual_encoded_cb
1456,1,1,10,2849.0,7,0.458333,3.779692,3.913101
2049,1,0,25,3.0,7,0.416667,3.737146,3.913101
644,1,1,80,2724.0,7,0.291667,3.779692,3.913101
346,0,0,11,538.0,7,0.160714,3.691904,3.744277
396,0,0,10,0.0,4,0.416667,3.691904,3.744277


In [6]:
tutor_df.iloc[1]

qual_encoded         1.000000
background_check     0.000000
num_subjects        25.000000
hours_tutoring       3.000000
edu_length           7.000000
mean_hours           0.416667
state_cb             3.737146
qual_encoded_cb      3.913101
Name: 2049, dtype: float64

In [17]:
loaded_model = pickle.load(open('model_base.sav', 'rb'))
loaded_encodings = pickle.load(open('cb_encoding.sav', 'rb'))
np.exp(loaded_model.predict(tutor_df.iloc[3])[0]).round()

35.0

In [ ]:
"Do you hold any degrees? Choose from the list."
"Do you have an approved federal background check?"
"Approximately how many hours have you tutored in the past year? Enter a number."
"Which subjects do you intend to teach? Choose as many as you like from the list."
"Where do you plan to teach? Pick a state."
"How many hours each week are you available to teach?"

In [14]:
degree_list = ['Bachelors','Masters','MD','MBA','PhD','J.D.','Other']

def qual_encode(qual):
    
    qual_list = ['Masters','MBA','PhD','J.D.']
    qual_encodings = {'Masters': 1, 'MBA': 1, 'PhD': 2, 'J.D.': 3}
    if qual in qual_list:
        return qual_encodings[qual]
    else:
        return 0

In [71]:
qual1 = 'Bachelors'
qual2 = 'Masters'
qual3 = 'PhD'
all_quals = [qual1,qual2,qual3]
qual_tot = 0

for qual in all_quals:
    qual_tot += qual_encode(qual)
    
back_check = 1
hours_tutored = 200
subject_list = ['1','2','4','4','5','6','7','8','5','2']
num_subjects = len(subject_list)
edu_length = 3*len(all_quals) + 1
state = 'CA'
hours = 10
mean_hours = hours/(24*7)

In [72]:
cats = pd.Series({'state':state,'qual_encoded':qual_tot})
user_cats = loaded_encodings.transform(cats.to_frame().T).add_suffix('_cb')
user_input = pd.Series({'qual_encoded':qual_tot, 'background_check':back_check, 'num_subjects':num_subjects,
                          'hours_tutoring':hours_tutored, 'edu_length':edu_length, 'mean_hours': mean_hours})
u_input = user_input.to_frame().T.join(user_cats)
u_input

,qual_encoded,background_check,num_subjects,hours_tutoring,edu_length,mean_hours,state_cb,qual_encoded_cb
0,3.0,1.0,10.0,200.0,10.0,0.059524,3.871119,4.277966


In [73]:
np.exp(loaded_model.predict(u_input.values))[0].round()

52.0

In [75]:
edu = np.array([1,2,0])

In [77]:
edu[edu!=0]

array([1, 2])